In [27]:
from groq import Groq
from dotenv import load_dotenv
from langchain_core.pydantic_v1 import BaseModel, Field
import os 
import subprocess
import sys
from textwrap import dedent
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,CommaSeparatedListOutputParser
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
load_dotenv()
api_key =  os.environ.get('GROQ_API_KEY')

In [28]:
# set up groq client:
client = Groq(
    api_key=api_key,
)
llm = ChatGroq(
            temperature=0,
            model_name="mixtral-8x7b-32768",
            streaming=True
        )


------------------------------------------

In [29]:
# create a pydantic objecet for ouput parsing
class grade(BaseModel):
    """"
    Boolean value indicating whether the given user query requires python code to generate the better response     
    """
    code : bool = Field(description="Validation score 'True' or 'False'")
    
parser =  JsonOutputParser(pydantic_object=grade)
query = "can u write a story for me?" 
prompt = PromptTemplate(
    template=dedent(
    """"
    Based on the user query, determine whether it is beneficial to use python code to generate a better response.
    -Respond True if it is beneficial to use python code to generate a better response, otherwise respond False.
    -Is there any calculation or similar needed to be performed repond True 
    -------- User query------
    {query}
    ------------------------
    Provide the Boolean value as a JSON,
    DO NOT provide any explanation and use these instructons to format the output  \n: 
    {format_instructions}"""),
      
input_variables=["query"],
partial_variables={"format_instructions": parser.get_format_instructions()})


In [30]:
chain = prompt | llm | parser
response = chain.invoke(
    {
        "query": query,
        "fromat_instructions":parser.get_format_instructions()
    }
)

In [33]:
response['code']

False

---------------------------------------------

In [15]:
system_message_1 = "You are a super creative expert in python , user task to provide a best python code to accomplish the given user goal "
human_message_1 =dedent(
    """"
    Based on the user query, provide the python code to solve the given problem.
    -------- User query------
    {user_goal}
    ------------------------
    - DO NOT provide any explanation just provide the python code 
    - Dont add any preamble in the response \n:""" )

system_message_2 = "You are a expert in python , user task to remove all text that are not needed to run the python code: "

human_message_2 =dedent(
    """"
    Based on the provided python code, remove all text that are not needed to run the python code.
    - check for any syntax errors, if any fix those errors 
    -------- code ------
    {code}
    ------------------------
    - DO NOT provide any explanation just provide the python code 
    - just provide the python code 
    - Dont add any preamble in the response
    - provide the complete code \n:""" )


In [22]:
# creating a funtion that takes a user goal as input and provide the python code to solve that 



def initial_code_creation(user_goal:str):
    prompt = ChatPromptTemplate.from_messages([("system", system_message_1), ("human", human_message_1)])
    chain = prompt | llm 
    response = chain.invoke(
    {
        "user_goal": user_goal
    })
    return response.content

def code_preproceess(initial_code:str):
    prompt = ChatPromptTemplate.from_messages([("system", system_message_2), ("human", human_message_2)])
    chain = prompt | llm 
    response = chain.invoke(
    {
        "code": initial_code
    })
    return response.content


def clean_code(code:str):
    clean_code = code.replace("```python","").replace("```","")
    return clean_code

def save_code_to_file(code: str, file_name: str = "generated_code.py"):
    with open(file_name, "w") as file:
        file.write(code)
    return file_name

initial_code = initial_code_creation("provide a python code to print the 10 numbers from the fibonacci series that are divisible by 5")
code = code_preproceess(initial_code)
code = clean_code(code)
file_name = save_code_to_file(code)

print("Code saved to:", file_name)

Code saved to: generated_code.py


In [23]:
import subprocess

def execute_python_file_in_conda_env(file_path, conda_env_name):
    try:
        # Execute the Python file within the specified Conda environment
        result = subprocess.run(["conda", "run", "-n", conda_env_name, "python", file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True)
        return True, result.stdout  # Execution successful, return stdout
    except subprocess.CalledProcessError as e:
        # If an error occurs during execution, return False along with the error message
        
        # 
        #   need to call the self corrective agent 
        #
        
        return False, e.stderr

# Example usage:
file_path = "generated_code.py"  
conda_env_name = "genai"  
success, output = execute_python_file_in_conda_env(file_path, conda_env_name)
if success:
    print("Execution successful")
    print("Output:")
    print(output)
else:
    print("Execution failed with error:", output)


Execution successful
Output:
[0, 5, 55, 610, 6765, 75025, 832040, 9227465, 102334155, 1134903170]





--------------------

In [8]:
def fetch_from_script(file_path):
    try:
        with open(file_path, "r") as file:
            file_contents = file.read()
            return file_contents
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")

In [118]:
system_message_3 = "You are a expert python programmer "
human_message_3 =dedent(
    """"
    Based on the user code , You have to check import statements if there are any and list all the necessary libararies needed to installed
    - Dont add any preamble in the response 
    -------- code------
    {code}
    ------------------------
    - just find the libararies needed to install inorder to run the script
    - Dont add any preamble in the response 
    - if there is any libraries need to be installed reespond in the following format
    \n:
    {format_instructions}""" )

In [134]:
# findall depencies and install those dependencies 

def findall_dependencies(file_path):
    try:
        with open(file_path, "r") as file:
            file_contents = file.read()
        
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    
    output_parser = CommaSeparatedListOutputParser()
    prompt = ChatPromptTemplate.from_messages([("system", system_message_3), ("human", human_message_3)])
    chain = prompt | llm | output_parser
    response = chain.invoke(
        {
            "code": file_contents,
            "format_instructions":output_parser.get_format_instructions()
        }
    )
    return response

def find_import_statements(script):
    # Split the script into lines
    lines = script.split('\n')
    
    # Check each line for import statements
    for line in lines:
        if line.strip().startswith("import ") or line.strip().startswith("from "):
            return True  # Found an import statement
    return False  # No import statements found


def create_txt_file(values: list, file_name: str = "requirements.txt"):
    # Split the comma-separated values
    # value_list = values.split(",")

    # Write each value to the text file
    with open(file_name, "w") as file:
        for value in values:
            file.write(value.strip() + "\n")

    print(f"Values written to {file_name}")
    
def install_packages_from_requirements(requirements_file='requirements.txt',conda_env_name='genai' ):
    try:
        # Create a new Conda environment from the requirements file
        subprocess.run(["conda", "create", "--name", conda_env_name, "--file", requirements_file, "-y"], check=True)
        
        print(f"Conda environment '{conda_env_name}' created successfully from '{requirements_file}'.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while creating Conda environment: {e}")

# wrapper function to find dependencies:
def dependencies(file_path):
    try:
        with open(file_path, "r") as file:
            file_contents = file.read()
        
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        
    if find_import_statements(file_contents):
        dependencies = findall_dependencies(file_path)
        create_txt_file(dependencies)
        install_packages_from_requirements()
    else:
        print("No import statements found in the script.")
        
# testing:
dependencies("generated_code.py")


Values written to requirements.txt


In [1]:
import subprocess

def install_requirements_in_env(conda_env_name, requirements_file):
    try:
        # Install packages from requirements.txt into the specified Conda environment
        subprocess.run(["conda", "install", "--name", conda_env_name, "--file", requirements_file], check=True)
        
        print(f"Requirements installed successfully into Conda environment '{conda_env_name}'.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while installing requirements: {e}")

# Example usage:
conda_env_name = "genai"  # Replace with your desired Conda environment name
requirements_file = "requirements.txt"  # Replace with the path to your requirements.txt file
install_requirements_in_env(conda_env_name, requirements_file)


Requirements installed successfully into Conda environment 'genai'.


In [33]:
## fix dependencies sorted 

# def install_missing_module( module_name,conda_env_name = "genai"):
#     try:
#         print(module_name)
#         # Install the missing module into the specified Conda environment
#         subprocess.run(["conda", "install", "-n", conda_env_name, "-c", "conda-forge", module_name, "-y"], check=True)
#         print(f"Module '{module_name}' installed successfully into Conda environment '{conda_env_name}'.")
#     except subprocess.CalledProcessError as e:
#         print(f"Error occurred while installing module '{module_name}': {e}")



def pip_install_missing_module(module_name, conda_env_name="genai"):
    try:
        # Install the missing module into the specified Conda environment using pip
        subprocess.run(["conda", "run", "-n", conda_env_name, "pip", "install", module_name], check=True)
        print(f"Module '{module_name}' installed successfully into Conda environment '{conda_env_name}'.")
        with open("requirements.txt", "a") as file:
            file.write(module_name + "\n")
        print(f"Module '{module_name}' appended to 'requirements.txt'.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while installing module '{module_name}': {e}")







def execute_python_file_in_conda_env(file_path, conda_env_name= 'genai'):
    try:
        # Execute the Python file within the specified Conda environment
        result = subprocess.run(["conda", "run", "-n", conda_env_name, "python", file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True)
        return True, result.stdout  # Execution successful, return stdout
    except subprocess.CalledProcessError as e:
        error_message = e.stderr
        # If module not found error, install the missing module and retry execution
        if "ModuleNotFoundError" in error_message:
            module_name = error_message.split("'")[1]
            print(f"Module '{module_name}' not found in Conda environment '{conda_env_name}'. Installing...")
            pip_install_missing_module(module_name)
            # Retry execution after installing the missing module
            return execute_python_file_in_conda_env(file_path)
        else:
            # If an error occurs during execution other than module not found, return False along with the error message
            return False, error_message

# Example usage:
file_path = "main.py"  # Replace with the path to your Python file
conda_env_name = "genai"  # Replace with the name of your Conda environment
def fix_dependencies(file_path, conda_env_name="genai"):
    success, output = execute_python_file_in_conda_env(file_path)
    if success:
        print("all dependencies are installed")
    else:   
        print("Execution failed with error:", output)

fix_dependencies("generated_code.py")

all dependencies are installed


-------------

    

In [11]:
user_goal = "provide a python code to print the 10 numbers from the fibonacci series that are divisible by 5"

In [12]:
system_message_4 = "You are a expert python programmer you're task to fix the errors in the code"
human_message_4 =dedent(
    """"
    Based on the user code , and the error message you have to fix the errors in the code
    - fix the errors and provide the corrected code 
    user goal:
    {user_goal}
    code:
    {code}
    error:
    {error_msg}
    -dont add any preamble in the response
    -fix the error and provide the corrected code
    - do not add any explanation just provide the corrected code\n:""" )

In [26]:
def execute_python_file_in_conda_env(file_path, conda_env_name):
    try:
        # Execute the Python file within the specified Conda environment
        result = subprocess.run(["conda", "run", "-n", conda_env_name, "python", file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True)
        print("stdout:", result.stdout)  # Print stdout
      #  print("stderr:", result.stderr)  # Print stderr
        return True, result.stdout  # Execution successful, return stdout
    except subprocess.CalledProcessError as e:
        # If an error occurs during execution, return False along with the error message
        return False, e.stderr


def self_correct_code(code,error):
    prompt = ChatPromptTemplate.from_messages([("system", system_message_4), ("human", human_message_4)])
    chain = prompt | llm 
    response = chain.invoke(
    {
        "user_goal": user_goal,
        "code":code,
        "error_msg":error
    })
    return response.content
success,result = execute_python_file_in_conda_env("generated_code.py","genai")

if not success:
    print("Error:", result)
    code = fetch_from_script("generated_code.py")
    code = self_correct_code(code,result)
    code  = code_preproceess(code)
    code = clean_code(code)
    save_code_to_file(code,"generated_code.py")
else:
    print("Execution successful")
    print("Output:")
    print(result)

Error:   File "e:\code_files\final_projects\self_corrective_agent\generated_code.py", line 6

    if a % 5 =+ 0:

       ^^^^^

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='?


ERROR conda.cli.main_run:execute(49): `conda run python generated_code.py` failed. (See above for error)



In [34]:
import curses

In [36]:
def execute_python_file_in_conda_env(file_path, conda_env_name):
    try:
        # Execute the Python file within the specified Conda environment
        result = subprocess.run(["conda", "run", "-n", conda_env_name, "python", file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True)
        print("stdout:", result.stdout)  # Print stdout
      #  print("stderr:", result.stderr)  # Print stderr
        return True, result.stdout  # Execution successful, return stdout
    except subprocess.CalledProcessError as e:
        # If an error occurs during execution, return False along with the error message
        return False, e.stderr
execute_python_file_in_conda_env("generated_code.py","genai")

(False,
 '\nRedirection is not supported.\n\nERROR conda.cli.main_run:execute(49): `conda run python generated_code.py` failed. (See above for error)\n')

In [37]:
def execute_python_file_in_conda_env(file_path, conda_env_name):
    try:
        # Execute the Python file within the specified Conda environment
        result = subprocess.run(["conda", "run", "-n", conda_env_name, "python", file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True)
        print("stdout:", result.stdout)  # Print stdout
      #  print("stderr:", result.stderr)  # Print stderr
        return True, result.stdout  # Execution successful, return stdout
    except subprocess.CalledProcessError as e:
        # If an error occurs during execution, return False along with the error message
        return False, e.stderr
execute_python_file_in_conda_env("generated_code.py","genai")

(False,
 '\nRedirection is not supported.\n\nERROR conda.cli.main_run:execute(49): `conda run python generated_code.py` failed. (See above for error)\n')

In [49]:
import subprocess

def execute_python_file_in_conda_env(file_path, conda_env_name):
    try:
        # Activate the Conda environment
        activate_command = f"conda activate {conda_env_name}"
        subprocess.run(activate_command, shell=True, check=True)
        
        # Execute the Python file within the activated Conda environment using subprocess.Popen
        process = subprocess.Popen(["python", file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Read and print the output from stdout and stderr
        for line in process.stdout:
            print(line.decode().strip())  # Decode bytes to str and strip newline characters
        for line in process.stderr:
            print(line.decode().strip())  # Decode bytes to str and strip newline characters
        
        process.wait()  # Wait for the process to finish
        
        return True, None  # Execution successful
    except Exception as e:
        # If an error occurs during execution, return False along with the error message
        print("Error:", str(e))
        return False, None

# Example usage:
execute_python_file_in_conda_env("generated_code.py", "genai")



Redirection is not supported.


(True, None)